In [1]:
import pandas as pd
df = pd.read_csv('startup_funding.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009 entries, 0 to 3008
Data columns (total 10 columns):
Sr No                      3009 non-null int64
Date ddmmyyyy              3009 non-null object
Startup Name               3009 non-null object
Industry Vertical          2838 non-null object
SubVertical                2073 non-null object
City  Location             2829 non-null object
Investorsxe2x80x99 Name    2986 non-null object
InvestmentnType            3006 non-null object
Amount in USD              2049 non-null object
Remarks                    419 non-null object
dtypes: int64(1), object(9)
memory usage: 235.2+ KB


### Buat DataFrame yang mendeskripsikan Tiap Tiap kolom dari data Startup Funding

In [3]:
def column_desc(df):
    data = []
    for item in df.columns:
        temp = []
        temp.append(item)
        temp.append(str(df[item].dtype))
        temp.append(df[item].isnull().sum())
        temp.append(int(round(df[item].isnull().sum() / len(df) *100)))
        temp.append(df[item].nunique())
        temp.append(list(df[item].value_counts().index[0:2]))
        data.append(temp)
    return pd.DataFrame(data,columns=['DataFeature','TypeData','Null','NullPercent','Unique','Sample'])

In [4]:
df = pd.read_csv('startup_funding.csv')

In [5]:
column_desc(df)

,DataFeature,TypeData,Null,NullPercent,Unique,Sample
0,Sr No,int64,0,0,3009,"[2047, 1014]"
1,Date ddmmyyyy,object,0,0,1013,"[02/02/2015, 30/11/2016]"
2,Startup Name,object,0,0,2439,"[Ola Cabs, Swiggy]"
3,Industry Vertical,object,171,6,811,"[Consumer Internet, Technology]"
4,SubVertical,object,936,31,1912,"[Online Lending Platform, Online Pharmacy]"
5,City Location,object,180,6,109,"[Bangalore, Mumbai]"
6,Investorsxe2x80x99 Name,object,23,1,2385,"[Undisclosed Investors, Undisclosed investors]"
7,InvestmentnType,object,3,0,50,"[Private Equity, Seed Funding]"
8,Amount in USD,object,960,32,500,"[1,000,000, 500,000]"
9,Remarks,object,2590,86,72,"[Series A, Series B]"


In [6]:
list(df['Startup Name'].value_counts().index[0:2])

['Ola Cabs', 'Swiggy']

### Hapus Kolom Remarks

In [7]:
df.drop('Remarks' , axis=1,inplace=True)

### Hapus data yang kolom Industry Vertical dan Subverticalnya Kosong

In [8]:
df.dropna(subset=['Industry Vertical' , 'SubVertical'] , how='all',inplace=True)

### Hapus data yang Investorsxe2x80x99 Name atau InvestmentnType kosong atau City Location nya Kosong atau amount in usd nya kosong

In [9]:
df.dropna(subset=['Investorsxe2x80x99 Name','InvestmentnType','City  Location','Amount in USD'],how='any',inplace=True)

### Ubah Nama Kolom Investorsxe2x80x99 Name menjadi Investor

In [10]:
df.rename({'Investorsxe2x80x99 Name' : 'Investor'},axis=1,inplace=True)

### Isi Kolom Subvertical yang kosong dengan kolom Industry Verticalnya

In [11]:
def apply_sub_ind(x):
    if( pd.isna(x['SubVertical'])):
        return x['Industry Vertical']
    else:
        return x['SubVertical']

In [12]:
df['SubVertical'] = df[['SubVertical','Industry Vertical']].apply(apply_sub_ind,axis=1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1893 entries, 0 to 2837
Data columns (total 9 columns):
Sr No                1893 non-null int64
Date ddmmyyyy        1893 non-null object
Startup Name         1893 non-null object
Industry Vertical    1893 non-null object
SubVertical          1893 non-null object
City  Location       1893 non-null object
Investor             1893 non-null object
InvestmentnType      1893 non-null object
Amount in USD        1893 non-null object
dtypes: int64(1), object(8)
memory usage: 147.9+ KB


### Hapus data dengan kolom Amount in USD Mengandung Alphabet

In [14]:
def delete_alpha(x):
    return x.replace(',','')

In [15]:
df['Amount in USD'] = df['Amount in USD'].apply(delete_alpha)

In [16]:
df.drop(df[(df['Amount in USD'] == 'Undisclosed') | (df['Amount in USD'] == 'undisclosed')].index,inplace=True)

In [22]:
df.drop( df[df['Amount in USD'].str.contains('x')].index ,inplace=True)

In [17]:
df.columns[1:]

Index(['Date ddmmyyyy', 'Startup Name', 'Industry Vertical', 'SubVertical',
       'City  Location', 'Investor', 'InvestmentnType', 'Amount in USD'],
      dtype='object')

In [23]:
df[df['Amount in USD'] =='N/A']

,Sr No,Date ddmmyyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investor,InvestmentnType,Amount in USD


### Ubah type data kolom Amount in USD Menjadi int64 , ubah dulu semua koma di data menjadi string kosong dan ubah juga selain angka menjadi string kosong

In [32]:
df['Amount in USD'] =df['Amount in USD'].apply(lambda x : x.replace('+',''))

In [34]:
df['Amount in USD'] =df['Amount in USD'].apply(lambda x : x.replace('.00',''))

In [36]:
df['Amount in USD'] = df['Amount in USD'].astype('int64')

### Cari Lima Startup Yang Mendapat Pendanaan Paling Banyak

In [46]:
df.sort_values(by=['Amount in USD'],ascending=False)[['Startup Name','Amount in USD','Investor']].head()

,Startup Name,Amount in USD,Investor
25,Rapido Bike Taxi,3900000000,Westbridge Capital
616,Flipkart,2500000000,Softbank
795,Paytm,1400000000,SoftBank Group
931,Flipkart,1400000000,"Microsoft, eBay, Tencent Holdings"
2613,Flipkart.com,700000000,Steadview Capital and existing investors


In [40]:
df.groupby('Startup Name').sum().sort_values(by=['Amount in USD'],ascending=False).head()

,Sr No,Amount in USD
Startup Name,,
Flipkart,6007,4059700000
Rapido Bike Taxi,26,3900000000
Paytm,4959,2148950000
Ola,4618,984500000
Flipkart.com,2654,700000000


### Cari Lima Investor Yang mengeluarkan uang paling banyak

In [47]:
df.groupby('Investor').sum().sort_values(by=['Amount in USD'],ascending=False).head()

,Sr No,Amount in USD
Investor,,
Westbridge Capital,26,3900000000
Softbank,618,2500000000
SoftBank Group,2234,1462000000
"Microsoft, eBay, Tencent Holdings",940,1400000000
Steadview Capital and existing investors,2654,700000000


### Cari lima industri Vertical dengan Pendanaan Paling Banyak

In [49]:
df.groupby('Industry Vertical').sum().sort_values(by=['Amount in USD'],ascending=False).head()

,Sr No,Amount in USD
Industry Vertical,,
Consumer Internet,664222,6248891934
eCommerce,155252,5002533000
Transportation,155,3908584000
Technology,291098,2203457930
ECommerce,54427,1878797608
